# Environment Initialization

In [1]:
!pip3 install -Uq pip
!pip3 install -Uq google-cloud-vision
!pip3 install -q requests wget

In [2]:
import os
from google.cloud import vision
from google.protobuf import field_mask_pb2 as field_mask
from google.cloud import storage
from google.colab import auth
from google.protobuf.json_format import MessageToJson

import requests # to get image from the web
import shutil # to save it locally
from uuid import uuid4
import csv

In [3]:
PROJECT_ID = 'dubhacks-292818'
LOCATION_ID = 'us-east1'
PRODUCT_SET_ID = 'product_set'
PRODUCT_CATEGORY = 'apparel-v2'
IMAGE_BUCKET = 'dubhacks-ref-images'
VISION_BUCKET = 'dubhacks-vision-bucket'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/dubhacks-292818-2f87e46570da.json' # TODO
os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['LOCATION_ID'] = LOCATION_ID

# Google Vision API Python SDK

In [4]:
def create_product_set(
        project_id, location, product_set_id, product_set_display_name):
    """Create a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_set_display_name: Display name of the product set.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Create a product set with the product set specification in the region.
    product_set = vision.ProductSet(
            display_name=product_set_display_name)

    # The response is the product set with `name` populated.
    response = client.create_product_set(
        parent=location_path,
        product_set=product_set,
        product_set_id=product_set_id)

    # Display the product set information.
    print('Product set name: {}'.format(response.name))

## Import Product Sets

In [44]:
# Format CSV as https://cloud.google.com/vision/product-search/docs/csv-format
def import_product_sets(project_id, location, gcs_uri):
    """Import images of different products in the product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        gcs_uri: Google Cloud Storage URI.
            Target files must be in Product Search CSV format.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # Set the input configuration along with Google Cloud Storage URI
    gcs_source = vision.ImportProductSetsGcsSource(
        csv_file_uri=gcs_uri)
    input_config = vision.ImportProductSetsInputConfig(
        gcs_source=gcs_source)

    # Import the product sets from the input URI.
    response = client.import_product_sets(
        parent=location_path, input_config=input_config)

    print('Processing operation name: {}'.format(response.operation.name))
    # synchronous check of operation status
    result = response.result()
    print('Processing done.')

    for i, status in enumerate(result.statuses):
        print('Status of processing line {} of the csv: {}'.format(
            i, status))
        # Check the status of reference image
        # `0` is the code for OK in google.rpc.Code.
        if status.code == 0:
            reference_image = result.reference_images[i]
            print(reference_image)
        else:
            print('Status code not OK: {}'.format(status.message))

In [45]:
def list_product_sets(project_id, location):
    """List all product sets.
    Args:
        project_id: Id of the project.
        location: A compute region name.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # List all the product sets available in the region.
    product_sets = client.list_product_sets(parent=location_path)

    # Display the product set information.
    for product_set in product_sets:
        print('Product set name: {}'.format(product_set.name))
        print('Product set id: {}'.format(product_set.name.split('/')[-1]))
        print('Product set display name: {}'.format(product_set.display_name))
        print('Product set index time: ')
        print(product_set.index_time)

In [46]:
def get_product_set(project_id, location, product_set_id):
    """Get info about the product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)

    # Get complete detail of the product set.
    product_set = client.get_product_set(name=product_set_path)

    # Display the product set information.
    print('Product set name: {}'.format(product_set.name))
    print('Product set id: {}'.format(product_set.name.split('/')[-1]))
    print('Product set display name: {}'.format(product_set.display_name))
    print('Product set index time: ')
    print(product_set.index_time)

## List Products

In [47]:
def list_products(project_id, location):
    """List all products.
    Args:
        project_id: Id of the project.
        location: A compute region name.
    """
    client = vision.ProductSearchClient()

    # A resource that represents Google Cloud Platform location.
    location_path = f"projects/{project_id}/locations/{location}"

    # List all the products available in the region.
    products = client.list_products(parent=location_path)

    # Display the product information.
    for product in products:
        print('Product name: {}'.format(product.name))
        print('Product id: {}'.format(product.name.split('/')[-1]))
        print('Product display name: {}'.format(product.display_name))
        print('Product description: {}'.format(product.description))
        print('Product category: {}'.format(product.product_category))
        print('Product labels: {}\n'.format(product.product_labels))

In [48]:
def get_product(project_id, location, product_id):
    """Get information about a product.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_id: Id of the product.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product.
    product_path = client.product_path(
        project=project_id, location=location, product=product_id)

    # Get complete detail of the product.
    product = client.get_product(name=product_path)

    # Display the product information.
    print('Product name: {}'.format(product.name))
    print('Product id: {}'.format(product.name.split('/')[-1]))
    print('Product display name: {}'.format(product.display_name))
    print('Product description: {}'.format(product.description))
    print('Product category: {}'.format(product.product_category))
    print('Product labels: {}'.format(product.product_labels))

In [49]:
def list_products_in_product_set(
        project_id, location, product_set_id):
    """List all products in a product set.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product set.
    product_set_path = client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)

    # List all the products available in the product set.
    products = client.list_products_in_product_set(name=product_set_path)

    # Display the product information.
    count = 0
    for product in products:
        count += 1
        print('Product name: {}'.format(product.name))
        print('Product id: {}'.format(product.name.split('/')[-1]))
        print('Product display name: {}'.format(product.display_name))
        print('Product description: {}'.format(product.description))
        print('Product category: {}'.format(product.product_category))
        print('Product labels: {}'.format(product.product_labels))
    print("Total products: ", count)

## Search Images

In [50]:
def get_reference_image_uri(
        project_id, location, product_id):
    """List all images in a product.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_id: Id of the product.
    """
    client = vision.ProductSearchClient()

    # Get the full path of the product.
    product_path = client.product_path(
        project=project_id, location=location, product=product_id)

    # List all the reference images available in the product.
    reference_images = client.list_reference_images(parent=product_path)

    # Display the reference image information.
    for image in reference_images:
      return image.uri
    #     print('Reference image name: {}'.format(image.name))
    #     print('Reference image id: {}'.format(image.name.split('/')[-1]))
    #     print('Reference image uri: {}'.format(image.uri))
    #     print('Reference image bounding polygons: {}'.format(
    #         image.bounding_polys))

In [51]:
def get_image_metadata(bucket_name, blob_name):
    """Prints out a blob's metadata."""
    # bucket_name = 'your-bucket-name'
    # blob_name = 'your-object-name'

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    return blob.metadata

In [52]:
def convert(labelsStr):
  """
  Converts protobuf format of product labels returned by Vision API
  into a dictionary of key-value pairs
  """
  labelObj = {}
  labelsStr = ' '.join(labelsStr.strip()[1:-1].split('\n'))
  for label in labelsStr.split(','):
    key = label.split(' value: ')[0].split('key: ')[1].strip().strip('"')
    value = label.split(' value: ')[1].strip().strip('"')
    labelObj[key] = value
  return labelObj


In [53]:
def get_similar_products_uri(
        project_id, location, product_set_id, product_category,
        image_uri, filter):
    """Search similar products to image.
    Args:
        project_id: Id of the project.
        location: A compute region name.
        product_set_id: Id of the product set.
        product_category: Category of the product.
        image_uri: Cloud Storage location of image to be searched.
        filter: Condition to be applied on the labels.
        Example for filter: (color = red OR color = blue) AND style = kids
        It will search on all products with the following labels:
        color:red AND style:kids
        color:blue AND style:kids
    """
    # product_search_client is needed only for its helper methods.
    product_search_client = vision.ProductSearchClient()
    image_annotator_client = vision.ImageAnnotatorClient()

    # Create annotate image request along with product search feature.
    image_source = vision.ImageSource(image_uri=image_uri)
    image = vision.Image(source=image_source)

    # product search specific parameters
    product_set_path = product_search_client.product_set_path(
        project=project_id, location=location,
        product_set=product_set_id)
    product_search_params = vision.ProductSearchParams(
        product_set=product_set_path,
        product_categories=[product_category],
        filter=filter)
    image_context = vision.ImageContext(
        product_search_params=product_search_params)

    # Search products similar to the image.
    response = image_annotator_client.product_search(
        image, image_context=image_context)

    index_time = response.product_search_results.index_time
    print('Product set index time: ')
    print(index_time)

    results = response.product_search_results.results

    print('Search results:')
    for result in results:
        product = result.product

        print('Score(Confidence): {}'.format(result.score))
        # print('Image name: {}'.format(result.image))

        # print('Product name: {}'.format(product.name))
        # print('Product display name: {}'.format(
        #     product.display_name))
        # print('Product description: {}'.format(product.description))
        # # labels.append(str(product.product_labels))
        # labels = convert(str(product.product_labels))
        # print('Product labels:'.format(labels))
        # for key,val in labels.items():
        #   print(f'{key}: {val}')
        product_id = product.name.split('/')[-1]
        image_uri = get_reference_image_uri(PROJECT_ID, LOCATION_ID, product_id)
        blob_name = image_uri.split('/')[-1]
        meta = get_image_metadata(IMAGE_BUCKET, blob_name)
        print("Product Info: ", meta)

## Examples

In [13]:
# Create one product set
create_product_set(PROJECT_ID, LOCATION_ID, 'test_product_set_id2', 'test_product_set2')
# Create everything (product sets, products, and reference images) based on csv file
import_product_sets(PROJECT_ID, LOCATION_ID, "gs://cloud-samples-data/vision/product_search/product_catalog.csv")
# List all product sets
list_product_sets(PROJECT_ID, LOCATION_ID)
# Get one product set
get_product_set(PROJECT_ID, LOCATION_ID, PRODUCT_SET_ID)
# Get one product
get_product(PROJECT_ID, LOCATION_ID, 'product_id78')
# Search for products
get_similar_products_uri(PROJECT_ID, LOCATION_ID, PRODUCT_SET_ID, 'apparel-v2', 'https://cdn.shopify.com/s/files/1/0017/2100/8243/products/CWJ-1_BROWN_61e80a3c-a589-43ab-89e7-1d8c69e04d25_2000x.jpg?v=1589312314', '')

AlreadyExists: ignored

# Scraping to Products Pipeline

## Download image from uri


In [19]:
# https://towardsdatascience.com/how-to-download-an-image-using-python-38a75cfa21c
def downloadImage(image_url):
  try:
    ## Set up the image URL and filename
    extension = '.' + image_url.split("/")[-1].split('?')[0].split('.')[-1]
    filename = 'image' + extension

    # Open the url image, set stream to True, this will return the stream content.
    r = requests.get(image_url, stream = True)

    # Check if the image was retrieved successfully
    if r.status_code == 200:
      # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
      r.raw.decode_content = True
      
      # Open a local file with wb ( write binary ) permission.
      with open(filename,'wb') as f:
        shutil.copyfileobj(r.raw, f)
    return filename
  except:
    print('Image Couldn\'t be retreived: ', image_url)

## Upload image to Google Cloud Storage

In [24]:
# https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
def set_image_metadata(bucket, image_blob_name, metadata=''):
  """Uploads a file to the bucket."""
  # bucket_name = "your-bucket-name"
  # source_file_name = "local/path/to/file"
  # destination_blob_name = "storage-object-name"

  blob = bucket.blob(image_blob_name)
  blob.metadata = metadata
  blob.patch()

In [21]:
# https://cloud.google.com/storage/docs/uploading-objects#storage-upload-object-python
def upload_blob(bucket, source_file_name, destination_blob_name):
  """Uploads a file to the bucket."""
  # bucket_name = "your-bucket-name"
  # source_file_name = "local/path/to/file"
  # destination_blob_name = "storage-object-name"

  blob = bucket.blob(destination_blob_name)

  blob.upload_from_filename(source_file_name)
  blob.patch()


## Read recommendations file


`recommendations.csv` file format:

| Name                           | Price | Link                          | Image                       | Brand          |
|--------------------------------|-------|-------------------------------|-----------------------------|----------------|
| BOTANICAL REVERSIBLE FACE MASK | 50.0  | https://biancaspender.com/... | https://cdn.shopify.com/... | Bianca Spender |

In [26]:
visionProductsList = []
PRODUCT_SET_ID = 'products_set_labelled'
IMAGE_BUCKET = 'dubhacks-images-labelled-bucket'
storage_client = storage.Client()
bucket = storage_client.bucket(IMAGE_BUCKET)
with open('./recommendations.csv') as f:
  for line in f:
    try:
      line = line.strip().split(',')
      
      # Upload image to GCS
      imageFile = downloadImage(line[3])
      extension = '.' + imageFile.split('.')[-1]
      imageId = str(uuid4())
      newImageFile = imageId+extension
      upload_blob(bucket, imageFile, newImageFile)
      query_labels = str(detect_labels_uri('gs://'+IMAGE_BUCKET+'/'+newImageFile))
      labels = {'name': line[0],
                'price': line[1],
                'link': line[2],
                'imageLink': line[3],
                'brand': line[4],
                'labels': query_labels}
      set_image_metadata(bucket, newImageFile, labels)

      productId = str(uuid4())
      visionProductsList.append(['gs://'+IMAGE_BUCKET+'/'+newImageFile, imageId, PRODUCT_SET_ID, productId, PRODUCT_CATEGORY, '', '', ''])
    except Exception as e:
      print(e)























































































































































































































Image Couldn't be retreived:  £95.00
'NoneType' object has no attribute 'split'


Image Couldn't be retreived:  £75.00
'NoneType' object has no attribute 'split'




















## Upload products file to GCS

In [31]:
CATALOG = 'vision_product_search_product_catalog.csv'
bucket = storage_client.bucket(VISION_BUCKET)
with open(CATALOG, 'w') as file:
  write = csv.writer(file) 
  write.writerows(visionProductsList)

upload_blob(bucket, CATALOG, CATALOG)

234

## Import Vision Product Set

In [34]:
import_product_sets(PROJECT_ID, LOCATION_ID, "gs://"+VISION_BUCKET+"/"+CATALOG)

Processing operation name: projects/dubhacks-292818/locations/us-east1/operations/4eff0978e64711dd
Processing done.
Status of processing line 0 of the csv: 
name: "projects/dubhacks-292818/locations/us-east1/products/b00a7c9d-6cfd-406d-9d11-3ef61b0c0574/referenceImages/d2f2cc31-5645-4c83-968d-9f877cba8379"
uri: "gs://dubhacks-images-labelled-bucket/d2f2cc31-5645-4c83-968d-9f877cba8379.jpg"

Status of processing line 1 of the csv: 
name: "projects/dubhacks-292818/locations/us-east1/products/e1e96674-ebfd-495f-ac15-f79d6c1e96b1/referenceImages/19838cf6-da66-4cff-b4cb-43d2294a21f5"
uri: "gs://dubhacks-images-labelled-bucket/19838cf6-da66-4cff-b4cb-43d2294a21f5.jpg"

Status of processing line 2 of the csv: 
name: "projects/dubhacks-292818/locations/us-east1/products/57c94cc7-2631-41f0-8660-e45e13369711/referenceImages/6480e02a-0f27-4cbf-bd1d-55a0b6999453"
uri: "gs://dubhacks-images-labelled-bucket/6480e02a-0f27-4cbf-bd1d-55a0b6999453.jpg"

Status of processing line 3 of the csv: 
name: "pr

# Search

In [42]:
image='https://cdn.shopify.com/s/files/1/1004/1966/products/STORYmfg_Vertical03_900x.jpg?v=1573147039'
get_similar_products_uri(PROJECT_ID, LOCATION_ID, PRODUCT_SET_ID, 'apparel-v2', image, '')

Product set index time: 
None
Search results:


# Image Classification

In [11]:
def detect_labels_uri(uri):
    """Detects labels in the file located in Google Cloud Storage or on the
    Web."""
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.label_detection(image=image)
    labels = response.label_annotations

    # for label in labels:
    #     print(label.description)
    print(response.error)
    if response.error.message:
        raise Exception(
            '{}'+uri+'\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))
    return [label.description for label in labels]

In [12]:

# print(detect_labels_uri('https://cdn.shopify.com/s/files/1/0373/2642/2152/products/IMG_2918_420x.jpg?v=1602820943'))
print(detect_labels_uri('https://images.squarespace-cdn.com/content/v1/55b6320ce4b074d8cb3522bd/1535749993420-PG23QQ1BG9Z220WW0AB3/ke17ZwdGBToddI8pDm48kIpS5DwFod8iuU2kzNJHUwEUqsxRUqqbr1mOJYKfIPR7LoDQ9mXPOjoJoqy81S2I8PaoYXhp6HxIwZIk7-Mi3Tsic-L2IOPH3Dwrhl-Ne3Z2OBFoxzNVOZI4qzc71ghGbt3OHMpRxvPs6IXctwhq0wtbTvJybZOtvbWRiFosmN10/varanasi-blue-mulberry-silk-robe.jpg'))

code: 14
message: "We can not access the URL currently. Please download the content and pass it in."



Exception: ignored

# Other

In [ ]:
labels

['[key: "name"\nvalue: "Reverse Terry Popover in Amber"\n, key: "price"\nvalue: "$110"\n, key: "link"\nvalue: "https://shop.kotn.com//collections/mens-lounge/products/reverse-terry-popover-in-amber"\n, key: "brand"\nvalue: "Kotn"\n]',
 '[key: "name"\nvalue: "Reverse Terry Popover in Alabaster"\n, key: "price"\nvalue: "$110"\n, key: "link"\nvalue: "https://shop.kotn.com//collections/mens-lounge/products/reverse-terry-popover-in-alabaster"\n, key: "brand"\nvalue: "Kotn"\n]',
 '[key: "name"\nvalue: "Reverse Terry Hoodie in Amber Multi"\n, key: "price"\nvalue: "$115"\n, key: "link"\nvalue: "https://shop.kotn.com//collections/mens-lounge/products/reverse-terry-hoodie-in-amber-multi"\n, key: "brand"\nvalue: "Kotn"\n]',
 '[key: "name"\nvalue: "Essential Sweatshirt in Deep Sea"\n, key: "price"\nvalue: "$80"\n, key: "link"\nvalue: "https://shop.kotn.com//collections/mens-lounge/products/mens-essential-sweatshirt-in-deep-sea"\n, key: "brand"\nvalue: "Kotn"\n]',
 '[key: "name"\nvalue: "Reverse Te

In [8]:
print(len(visionProductsList))
print(convert(labels[0]))

214


NameError: ignored

In [7]:
list_products_in_product_set(PROJECT_ID, LOCATION_ID, PRODUCT_SET_ID)

Product name: projects/dubhacks-292818/locations/us-east1/products/01cc0607-dc91-447c-8470-1f66c2124ea8
Product id: 01cc0607-dc91-447c-8470-1f66c2124ea8
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name: projects/dubhacks-292818/locations/us-east1/products/02025768-94f0-413e-a70c-b51e3027dc12
Product id: 02025768-94f0-413e-a70c-b51e3027dc12
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name: projects/dubhacks-292818/locations/us-east1/products/031a1773-9c91-4e1b-b91d-03744a89b41f
Product id: 031a1773-9c91-4e1b-b91d-03744a89b41f
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name: projects/dubhacks-292818/locations/us-east1/products/03f07686-d6c3-429d-8c11-276a4a3c0a11
Product id: 03f07686-d6c3-429d-8c11-276a4a3c0a11
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name

In [ ]:
with open('recommendations.csv', 'r') as f:
  count = 0
  for line in f:
    count += 1
  print(count)

212


In [54]:
list_product_sets(PROJECT_ID, LOCATION_ID)

Product set name: projects/dubhacks-292818/locations/us-east1/productSets/product_set
Product set id: product_set
Product set display name:  
Product set index time: 
2020-10-18 13:54:28.540731+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/product_set0
Product set id: product_set0
Product set display name:  
Product set index time: 
2020-10-18 13:54:28.540731+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/product_set_labelled
Product set id: product_set_labelled
Product set display name:  
Product set index time: 
2020-10-18 13:54:28.540731+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/products_set_labelled
Product set id: products_set_labelled
Product set display name:  
Product set index time: 
1970-01-01 00:00:00+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/recommendations
Product set id: recommendations
Product set display name:  
Product set index

In [ ]:
import_product_sets(PROJECT_ID, LOCATION_ID, "gs://dubhacks-vision-bucket/vision_product_search_product_catalog.csv")

Processing operation name: projects/dubhacks-292818/locations/us-east1/operations/b8e52b1293bdbe46
Processing done.
Status of processing line 0 of the csv: 
name: "projects/dubhacks-292818/locations/us-east1/products/c3487be6-e72b-4f34-9f3a-972cb85ef631/referenceImages/d4544baa-0f55-482a-9489-700403e2573d"
uri: "gs://dubhacks-images-bucket/d4544baa-0f55-482a-9489-700403e2573d.jpg"

Status of processing line 1 of the csv: 
name: "projects/dubhacks-292818/locations/us-east1/products/3422e2c2-e483-4c4e-bfdd-4bc7b1f9c166/referenceImages/b33682c2-13e6-44af-9db1-b6bf24cf1d4e"
uri: "gs://dubhacks-images-bucket/b33682c2-13e6-44af-9db1-b6bf24cf1d4e.jpg"

Status of processing line 2 of the csv: 
name: "projects/dubhacks-292818/locations/us-east1/products/37a3aa7d-9d2d-4809-b191-22cb1a136edc/referenceImages/5e1d79ad-ba68-437b-8075-9b9c0b360c38"
uri: "gs://dubhacks-images-bucket/5e1d79ad-ba68-437b-8075-9b9c0b360c38.jpg"

Status of processing line 3 of the csv: 
name: "projects/dubhacks-292818/loca

In [43]:
list_product_sets(PROJECT_ID, LOCATION_ID)

NameError: ignored

In [74]:

list_products_in_product_set(PROJECT_ID, LOCATION_ID, PRODUCT_SET_ID)

Product name: projects/dubhacks-292818/locations/us-east1/products/01cc0607-dc91-447c-8470-1f66c2124ea8
Product id: 01cc0607-dc91-447c-8470-1f66c2124ea8
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name: projects/dubhacks-292818/locations/us-east1/products/02025768-94f0-413e-a70c-b51e3027dc12
Product id: 02025768-94f0-413e-a70c-b51e3027dc12
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name: projects/dubhacks-292818/locations/us-east1/products/031a1773-9c91-4e1b-b91d-03744a89b41f
Product id: 031a1773-9c91-4e1b-b91d-03744a89b41f
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name: projects/dubhacks-292818/locations/us-east1/products/03f07686-d6c3-429d-8c11-276a4a3c0a11
Product id: 03f07686-d6c3-429d-8c11-276a4a3c0a11
Product display name:  
Product description: 
Product category: apparel-v2
Product labels: []
Product name

In [ ]:
def convert(labelsStr):
  labelsStr = ''.join(labelsStr.split('\n'))
  

Product set index time: 
2020-10-17 23:54:43.358749+00:00
Search results:
Score(Confidence): 0.1841801404953003
Image name: projects/dubhacks-292818/locations/us-east1/products/a6566600-e600-480b-9728-335476e3cb2e/referenceImages/b934f5dd-8747-4391-bc88-f6668f004271
Product name: projects/dubhacks-292818/locations/us-east1/products/a6566600-e600-480b-9728-335476e3cb2e
Product display name:  
Product description: 
Product labels:


AttributeError: ignored

In [69]:
get_reference_image_uri(PROJECT_ID, LOCATION_ID, 'f77076ce-4fb4-48f0-9e8a-9d0002a1f6c0')

'gs://dubhacks-image-bucket/842dbbbc-8553-43da-975c-84b9e9393747.jpg'

In [68]:
image_uri = get_reference_image_uri(PROJECT_ID, LOCATION_ID, 'f77076ce-4fb4-48f0-9e8a-9d0002a1f6c0')
blob_name = image_uri.split('/')[-1]
meta = get_image_metadata(IMAGE_BUCKET, blob_name)

AttributeError: ignored

In [67]:
meta = get_image_metadata(IMAGE_BUCKET, '00763d99-a6aa-43f2-877b-15d29ad743bb.jpg')
meta

{'brand': 'Story mfg',
 'imageLink': 'https://cdn.shopify.com/s/files/1/1004/1966/products/GratefulTeeSS_Jackfruit-SUN-Tee_1_900x.jpg?v=1598540545',
 'link': 'https://www.storymfg.com/collections/shop-all/products/grateful-tee-ss-sun-tee',
 'name': 'Grateful Tee SS - Sun Tee',
 'price': '£95.00'}

In [91]:
def set_blob_metadata(bucket_name, blob_name):
    """Set a blob's metadata."""
    # bucket_name = 'your-bucket-name'
    # blob_name = 'your-object-name'

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)
    print("The metadata for the blob {} is {}".format(blob.name, blob.metadata))
    metadata = {'labels': "['Red', 'name', 'Test']"}
    blob.metadata = metadata
    blob.patch()

    print("The metadata for the blob {} is {}".format(blob.name, blob.metadata))


In [92]:
set_blob_metadata(IMAGE_BUCKET, '83de64c1-e3c4-4ea3-b5a4-72901e0f96ee.jpg')

The metadata for the blob 83de64c1-e3c4-4ea3-b5a4-72901e0f96ee.jpg is {'imageLink': 'https://cdn.shopify.com/s/files/1/0373/2642/2152/products/IMG_2918_420x.jpg?v=1602820943', 'price': '$50.00', 'name': 'BOTANICAL REVERSIBLE FACE MASK', 'link': 'https://biancaspender.com/collections/shop-all/products/botanical-reversible-face-mask', 'brand': 'Bianca Spender'}
The metadata for the blob 83de64c1-e3c4-4ea3-b5a4-72901e0f96ee.jpg is {'imageLink': 'https://cdn.shopify.com/s/files/1/0373/2642/2152/products/IMG_2918_420x.jpg?v=1602820943', 'price': '$50.00', 'name': 'BOTANICAL REVERSIBLE FACE MASK', 'link': 'https://biancaspender.com/collections/shop-all/products/botanical-reversible-face-mask', 'brand': 'Bianca Spender', 'labels': "['Red', 'name', 'Test']"}


Blob: 2af60309-6dd9-411c-9e13-074736e579d8.jpg
Bucket: test-dubhacks-bucket
Storage class: STANDARD
ID: test-dubhacks-bucket/2af60309-6dd9-411c-9e13-074736e579d8.jpg/1602992807962386
Size: 30305 bytes
Updated: 2020-10-18 03:46:47.962000+00:00
Generation: 1602992807962386
Metageneration: 1
Etag: CJLOlcedvewCEAE=
Owner: None
Component count: None
Crc32c: q/6AKg==
md5_hash: aqCw89/ZkhXrgxssWYHV6Q==
Cache-control: None
Content-type: image/jpeg
Content-disposition: None
Content-encoding: None
Content-language: None
Metadata: None


AttributeError: ignored

In [32]:
def blob_metadata(bucket_name, blob_name):
    """Prints out a blob's metadata."""
    # bucket_name = 'your-bucket-name'
    # blob_name = 'your-object-name'

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.get_blob(blob_name)

    print("Blob: {}".format(blob.name))
    print("Bucket: {}".format(blob.bucket.name))
    print("Storage class: {}".format(blob.storage_class))
    print("ID: {}".format(blob.id))
    print("Size: {} bytes".format(blob.size))
    print("Updated: {}".format(blob.updated))
    print("Generation: {}".format(blob.generation))
    print("Metageneration: {}".format(blob.metageneration))
    print("Etag: {}".format(blob.etag))
    print("Owner: {}".format(blob.owner))
    print("Component count: {}".format(blob.component_count))
    print("Crc32c: {}".format(blob.crc32c))
    print("md5_hash: {}".format(blob.md5_hash))
    print("Cache-control: {}".format(blob.cache_control))
    print("Content-type: {}".format(blob.content_type))
    print("Content-disposition: {}".format(blob.content_disposition))
    print("Content-encoding: {}".format(blob.content_encoding))
    print("Content-language: {}".format(blob.content_language))
    print("Metadata: {}".format(blob.metadata))
    print("Temporary hold: ", "enabled" if blob.temporary_hold else "disabled")
    print(
        "Event based hold: ",
        "enabled" if blob.event_based_hold else "disabled",
    )
    if blob.retention_expiration_time:
        print(
            "retentionExpirationTime: {}".format(
                blob.retention_expiration_time
            )
        )


In [65]:
blob_metadata(IMAGE_BUCKET, '00763d99-a6aa-43f2-877b-15d29ad743bb.jpg')

{'brand': 'Story mfg',
 'imageLink': 'https://cdn.shopify.com/s/files/1/1004/1966/products/GratefulTeeSS_Jackfruit-SUN-Tee_1_900x.jpg?v=1598540545',
 'link': 'https://www.storymfg.com/collections/shop-all/products/grateful-tee-ss-sun-tee',
 'name': 'Grateful Tee SS - Sun Tee',
 'price': '£95.00'}

In [75]:
list_product_sets(PROJECT_ID, LOCATION_ID)


Product set name: projects/dubhacks-292818/locations/us-east1/productSets/product_set
Product set id: product_set
Product set display name:  
Product set index time: 
1970-01-01 00:00:00+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/product_set0
Product set id: product_set0
Product set display name:  
Product set index time: 
2020-10-18 03:54:34.819937+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/recommendations
Product set id: recommendations
Product set display name:  
Product set index time: 
2020-10-18 03:54:34.819937+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/recommendations2
Product set id: recommendations2
Product set display name:  
Product set index time: 
2020-10-18 03:54:34.819937+00:00
Product set name: projects/dubhacks-292818/locations/us-east1/productSets/test_product_set_id
Product set id: test_product_set_id
Product set display name: test_product_set
Product set in